In [6]:
from langchain_openai import AzureChatOpenAI
from util.config import load_configuration
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [7]:
PROP_FILE = 'app.properties'
configs = load_configuration(PROP_FILE)

API_TYPE = configs.get("API_TYPE").data
API_VERSION = configs.get("API_VERSION").data
API_KEY = configs.get("API_KEY").data
API_BASE = configs.get("API_BASE").data
LLM_ENGINE_GPT35=configs.get("LLM_ENGINE_GPT35").data
LLM_ENGINE_GPT4=configs.get("LLM_ENGINE_GPT4").data
LLM_ENGINE_GPT35_16K=configs.get("LLM_ENGINE_GPT35_16K").data
ADA_MODEL=configs.get("ADA_MODEL").data

In [9]:
llm = AzureChatOpenAI(
    azure_endpoint=API_BASE,
    api_key=API_KEY,
    azure_deployment=LLM_ENGINE_GPT35_16K,
    openai_api_version=API_VERSION,
)

In [10]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

output_parser = StrOutputParser()

In [11]:
chain = prompt | llm | output_parser

In [12]:
chain.invoke({"input": "how can langsmith help with testing?"})

'Langsmith can greatly assist in the testing process by providing various features and functionalities that aid in creating and executing tests. Here are a few ways Langsmith can help with testing:\n\n1. Test case management: Langsmith allows you to create and organize test cases in a structured manner. You can easily define test steps, expected results, and any preconditions or dependencies for each test case. This helps ensure comprehensive test coverage and makes it easier to track progress.\n\n2. Test automation: Langsmith supports test automation frameworks, allowing you to write and execute automated tests. You can use popular testing frameworks like Selenium or Appium to automate web or mobile application testing. With automation, you can increase test efficiency, reduce manual effort, and improve overall test coverage.\n\n3. Test data management: Langsmith provides capabilities for managing test data effectively. You can define and maintain test data sets, create data-driven te

In [13]:
## Retrieval Chain, provide additional context to the LLM

from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [14]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=API_BASE,
    azure_deployment=ADA_MODEL,
     api_key=API_KEY,
    openai_api_version=API_VERSION,
)

In [15]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [16]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [17]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [18]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

LangSmith can help with testing in several ways. First, it allows developers to create datasets that contain inputs and reference outputs, which can be used to run tests on their LLM applications. These test cases can be uploaded, created on the fly, or exported from application traces. LangSmith also provides the ability to run custom evaluations, both LLM and heuristic-based, to score test results.

In addition, LangSmith offers a comparison view that allows developers to see the results of different configurations on the same datapoints side-by-side. This is useful for tracking and diagnosing regressions in test scores across multiple revisions of an application.

Furthermore, LangSmith provides a playground environment for rapid iteration and experimentation. Developers can quickly test out different prompts and models, and every playground run is logged in the system and can be used to create test cases or compare with other runs.

Overall, LangSmith supports the testing process b

In [19]:
## Conversation Retrieval Chain

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [24]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]

In [25]:
answer=retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

for d in answer:
    print(d.metadata['title'])

LangSmith User Guide | ğŸ¦œï¸�ğŸ› ï¸� LangSmith
LangSmith User Guide | ğŸ¦œï¸�ğŸ› ï¸� LangSmith
LangSmith User Guide | ğŸ¦œï¸�ğŸ› ï¸� LangSmith
LangSmith User Guide | ğŸ¦œï¸�ğŸ› ï¸� LangSmith


In [26]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [27]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
response=retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})
print(response["answer"])

LangSmith provides several features that can help test your LLM applications:

1. Initial Test Set: LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, to run tests on their LLM applications. You can upload test cases in bulk, create them on the fly, or export them from application traces. LangSmith also supports custom evaluations to score test results.

2. Comparison View: When prototyping different versions of your applications and making changes, it's important to see if you've regressed with respect to your initial test cases. LangSmith offers a user-friendly comparison view for test runs, allowing you to track and diagnose regressions in test scores across multiple revisions of your application.

3. Playground: LangSmith provides a playground environment for rapid iteration and experimentation. You can quickly test out different prompts and models, and every playground run is logged in the system, which can be used to create test

In [29]:
## Agent

from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [30]:
tools = [retriever_tool]

In [31]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# You need to set OPENAI_API_KEY environment variable or pass it as argument `api_key`.
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [32]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})



> Entering new AgentExecutor chain...
LangSmith can help test your LLM applications by providing valuable feedback and suggestions to improve your application. Here's how LangSmith can assist you:

1. Personal Statement Evaluation: LangSmith can review your personal statement and provide feedback on its structure, content, and overall effectiveness. It can suggest improvements to make your statement more compelling and persuasive.

2. Resume/CV Review: LangSmith can analyze your resume or CV and offer suggestions on how to highlight your academic and professional achievements effectively. It can provide guidance on formatting, content, and organization.

3. Letter of Recommendation Guidance: LangSmith can provide advice on selecting the right recommenders and guide you on how to request effective letters of recommendation. It can also offer tips on what information to provide to your recommenders to ensure strong and supportive letters.

4. Application Strategy: LangSmith can help yo

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'output': "LangSmith can help test your LLM applications by providing valuable feedback and suggestions to improve your application. Here's how LangSmith can assist you:\n\n1. Personal Statement Evaluation: LangSmith can review your personal statement and provide feedback on its structure, content, and overall effectiveness. It can suggest improvements to make your statement more compelling and persuasive.\n\n2. Resume/CV Review: LangSmith can analyze your resume or CV and offer suggestions on how to highlight your academic and professional achievements effectively. It can provide guidance on formatting, content, and organization.\n\n3. Letter of Recommendation Guidance: LangSmith can provide advice on selecting the right recommenders and guide you on how to request effective letters of recommendation. It can also offer tips on what information